In [1]:
from elasticsearch import Elasticsearch
from bs4 import BeautifulSoup
from datetime import datetime
from flask import Flask, render_template, request
import os


In [2]:

es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme': 'http'}], http_auth=("vijay", "password"))
es.info()

C:\Users\vijay\AppData\Local\Temp\ipykernel_80552\2575928368.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme': 'http'}], http_auth=("vijay", "password"))


ObjectApiResponse({'name': 'BOOK-GM09OBS762', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'vJxnvQteTUeURso4lvYqKw', 'version': {'number': '8.10.4', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': 'b4a62ac808e886ff032700c391f45f1408b2538c', 'build_date': '2023-10-11T22:04:35.506990650Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [3]:
directory_path = "Assignment 3 dataset/AP_DATA/ap89_collection"

#file_name = "ap890101" 

documents = list()

file_list = os.listdir(directory_path)

for file_name in file_list:

    file_path = os.path.join(directory_path, file_name)

    with open(file_path, "r") as file:

        tag_dict = dict()
        
        soup = BeautifulSoup(file, "xml")

        unique_tag_names = set(tag.name for tag in soup.find_all())

        for tag_name in unique_tag_names:

            content_list = soup.find_all(tag_name)

            for element in content_list:

                if tag_name in tag_dict:
                    
                    tag_dict[tag_name].append(element.text)

                else:

                    tag_dict[tag_name] = [element.text]
        
        
        documents.append(tag_dict)


print(documents[:10])    


[{'HEAD': ["You Don't Need a Weatherman To Know '60s Films Are Here", 'Eds: Also in Monday AMs report.'], 'FILEID': ['AP-NR-01-01-89 2358EST'], 'DOC': ["\n AP890101-0001 \nAP-NR-01-01-89 2358EST\nr a PM-APArts:60sMovies     01-01 1073\nPM-AP Arts: 60s Movies,1100\nYou Don't Need a Weatherman To Know '60s Films Are Here\nEds: Also in Monday AMs report.\nBy HILLEL ITALIE\nAssociated Press Writer\nNEW YORK (AP) \n\n   The celluloid torch has been passed to a new\ngeneration: filmmakers who grew up in the 1960s.\n   ``Platoon,'' ``Running on Empty,'' ``1969'' and ``Mississippi\nBurning'' are among the movies released in the past two years from\nwriters and directors who brought their own experiences of that\nturbulent decade to the screen.\n   ``The contemporaries of the '60s are some of the filmmakers of\nthe '80s. It's natural,'' said Robert Friedman, the senior vice\npresident of worldwide advertising and publicity at Warner Bros.\n   Chris Gerolmo, who wrote the screenplay for ``Missis

In [4]:
def index_documents(index, id, document):
    
    resp = es.index(index=index, id=id, document=document)
    print(resp['result'])


In [5]:
for i in range(len(documents)):
    
    es.index(index = "my_index", id = i + 1, document = documents[i])


In [6]:
resp = es.get(index="my_index", id=1)
print(resp['_source'])

{'HEAD': ["You Don't Need a Weatherman To Know '60s Films Are Here", 'Eds: Also in Monday AMs report.'], 'FILEID': ['AP-NR-01-01-89 2358EST'], 'DOC': ["\n AP890101-0001 \nAP-NR-01-01-89 2358EST\nr a PM-APArts:60sMovies     01-01 1073\nPM-AP Arts: 60s Movies,1100\nYou Don't Need a Weatherman To Know '60s Films Are Here\nEds: Also in Monday AMs report.\nBy HILLEL ITALIE\nAssociated Press Writer\nNEW YORK (AP) \n\n   The celluloid torch has been passed to a new\ngeneration: filmmakers who grew up in the 1960s.\n   ``Platoon,'' ``Running on Empty,'' ``1969'' and ``Mississippi\nBurning'' are among the movies released in the past two years from\nwriters and directors who brought their own experiences of that\nturbulent decade to the screen.\n   ``The contemporaries of the '60s are some of the filmmakers of\nthe '80s. It's natural,'' said Robert Friedman, the senior vice\npresident of worldwide advertising and publicity at Warner Bros.\n   Chris Gerolmo, who wrote the screenplay for ``Mississ

In [7]:
def search_query(q):
    # query = {
    #     "query": {
    #         "match": {
    #             "TEXT": q
    #         }
    #     }
    # }

    body = {
        "size": 20,
        "query": {
            "dis_max": {
                "queries": [
                    {"match": {"head": q}},
                    {"match": {"text": q}}
                ],
                "tie_breaker": 0.3
            }
        }
    }

    resp = es.search(index="my_index", body=body)

    results = []

    for hit in resp['hits']['hits']:
        
        title = (hit["_source"].get('HEAD') or ['No title available'])[0]
        author = (hit["_source"].get('BYLINE') or ['No author available'])[0]
        date = (hit["_source"].get('FILEID') or ['No date available'])[0]
        text = (hit["_source"].get('TEXT') or ['No text available'])[0]

        text.replace('\n', '')

        formatted_result = f"Title: {title}\nAuthor: {author}\nDate: {date}\n\n{text}\n\n"

        formatted_result = formatted_result.replace('\n', '<br>')

        results.append(formatted_result)

    return results


In [8]:
search_query("cat")

[]

In [10]:
app = Flask(__name__)


@app.route('/', methods=['GET', 'POST'])
def homepage():

    results = []

    if request.method == 'POST':

        query = request.form.get('search_query')
        
        results = search(query)
    
    return render_template('test.html', results=results)


def search(query):

    return search_query(query)


app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
